## Build a HybridUrb model from Delft3DFM - example of Eindhoven (In progress)

This notebook demonstrates how to prepare a **HybridUrb** model from **Delft3D FM** model using an example of Eindhoven.

### Preperation

> **NOTE**: 
All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

Running notebook requires the installation of dependencies which can be done by:

In [ ]:
# This should return Requirement already satisfied if installation is complete.
!pip install hybridurb[examples]

HybridUrb has a main dependency which is **hydromt-delft3dfm**. It provides api access Delft3D FM model files. 
The current example notebook needs newest development of hydromt-delft3dfm v0.2.0.dev0. We can install it from git repository:


In [ ]:
!pip install git+https://github.com/Deltares/hydromt_delft3dfm.git

Check if the HydroMT-Delft3D FM dependency is installed correctly:

In [ ]:
# this should return "dflowfm (hydromt_delft3dfm 0.2.0.dev0)" to build DFlowFM components of Delft3D FM
!hydromt --models

After that checks out, we can start python and load all the neccessary packages:

In [ ]:
import logging
from pathlib import Path
from typing import Optional

import geopandas as gpd
import numpy as np
from pyproj import CRS
import matplotlib.pyplot as plt

import hybridurb.networksutils.delft3dfm_utils as delft3dfm_utils
from hydromt_delft3dfm import DFlowFMModel

### Delft3D FM model for Eindhoven

First, let's have a quick look at the Eindhoven model. 

In [ ]:
# Define the root path where the project files are located
model_root = Path(r"..\data\Eindhoven\Delft3DFM")

# Define the filename of the Delft3D FM model configuration file (*.mdu) as relative path to the model root.
mdu_fn = "dflowfm\FlowFM.mdu"

# read the delft3dfm model (there will be an error message but we can ignore that)
model = DFlowFMModel(root=model_root, config_fn=mdu_fn, mode="r+", crs=28992)

# read the clip region
clip_region = gpd.read_file(r"..\data\Eindhoven\region.geojson")

# make a plot of the model and the region of interest
fig, ax = plt.subplots()
model.geoms["branches"].plot(ax=ax, color="tab:blue", zorder=1)
clip_region.plot(ax=ax, color = 'yellow', alpha = 0.5)

### Model setup configuration 

The model configuration file (YML) contains the model setup configuration and determines which methods are used to prepare the HybridUrb and in which order and optionally sets non-default arguments for each method. We have prepared an example yml-file for Eindhoven example which are available in the model repository [examples folder](https://github.com/xldeltares/hybridurb/tree/main/examples/Eindhoven). 

Each section, before indent, (e.g. get_geoms_from_model) corresponds to a model method. All model methods are explained in the [docs (model components)](https://xldeltares.github.io/hybridurb/api.html). The `global` section contains direct model initialisation properties. 

We will load the provided config.yaml file for Eindhoven for inspection:

In [ ]:
fn_yml = Path("Eindhoven/config.yml")
with open(fn_yml, "r") as f:
    txt = f.read()
print(txt)


Looking at this file, you see that we will prepare the following information for our model:

- global: Here we specify the root directry of where we work. The region argument determines the model bounds. Our model will be defined in the projected CRS EPSG 28992.
- get_geoms_from_model: We will derive the geometry files for setting up the networkm model of HybridUrb from an existing external model, in this case, Delft3D FM. The model is located in the model_root. For Delft3D FM model, an mdu file is needed as main configuration. This is provided as mdu_fn. 
- setup_networkmodel: We will setup the network model from the geometry derived from the previous step.   


> **NOTE**: 
  In the setup_networkmodel step, because the geometries are already derived, we dont need much information for this function. More arguments will be supported in the future.

### Set up a HybridUrb network

After geometries are derived, we can setup a HybirdUrb network using the functions below:

In [ ]:
from hybridurb.networksutils.network_from_delft3dfm import Delft3dfmNetworkWrapper
mywrapper = Delft3dfmNetworkWrapper(fn_yml)
mywrapper.get_geoms_from_model()
mywrapper.get_network_from_geoms()

> **NOTE**:
    The example above now uses python to directly interact with HybridUrb. That is because we have not yet finished the implementation of the entire HybridUrb model buildig functions. In the future, we will support [hydromt build](https://deltares.github.io/hydromt/latest/user_guide/model_build.html) methods.

### Setup a HybridUrb dataset

After the network is setup, we can setup datasets that facilates the training of the hybridurb machine learning model component. 

### Calibrate a HybridUrb model

### Visualize and/or inspect model schematization

### Run a HybridUrb model